# Рекомендательные системы

## Урок 2. Бейзлайны и item-item. Практическое задание.

In [1]:
import numpy as np
import pandas as pd
from scipy.sparse import bsr_matrix

# Детерминированные алгоритмы
from implicit.nearest_neighbours import ItemItemRecommender, CosineRecommender, TFIDFRecommender

In [2]:
%%time
data = pd.read_csv('retail_train.csv.zip')
data.sample(3, random_state=0)

Wall time: 3.6 s


,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
1549245,2102,33659352753,453,973086,1,2.00,450,-1.19,1718,65,0.0,0.0
1360383,763,33015750451,407,820895,1,1.39,31782,0.00,1123,59,0.0,0.0
827593,56,31070756490,274,10455921,1,2.99,439,0.00,1259,40,0.0,0.0


In [3]:
# train-test split
test_size_weeks = 3
data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

### Задание 1. Weighted Random Recommendation

Напишите код для случайных рекоммендаций, в которых вероятность рекомендовать товар прямо пропорциональна логарифму продаж
- Можно сэмплировать товары случайно, но пропорционально какому-либо весу
- Например, прямопропорционально популярности. Вес = log(sales_sum товара)

**Примечание** Сделал немного другую реализацию. Базовые рекомендации не зависят от `user_id`, применимы к любому количеству пользователей, их удобно реализовать в виде бесконечных генераторов.

In [4]:
# Случайные и взвешенные случайные рекоммендации
def gen_inf_random_rec(items, weights=None, n=5, random_state=None):
    """Generate infinite random weighted recommendations from items"""
    rng = np.random.default_rng(random_state)
    while True:
        yield rng.choice(items, p=weights, size=n, replace=False, shuffle=True).tolist()

In [5]:
# Применить логарифм непосредственно к sales_value не получится
np.any(data_train['sales_value'] == 0)

True

In [6]:
# Применим log(x+1)
item_weight_train = (
    data_train
    .groupby('item_id')
    .agg({'sales_value': 'sum'})
    .assign(w_logsum = lambda x: np.log1p(x['sales_value']))
    .assign(w_logsum = lambda x: x.w_logsum / np.sum(x.w_logsum))
    .assign(w_sum = lambda x: x['sales_value'] / np.sum(x['sales_value']))
)

item_weight_train.head(3)

,sales_value,w_logsum,w_sum
item_id,,,
25671,20.94,0.000013,2.969296e-06
26081,0.99,0.000003,1.403822e-07
26093,1.59,0.000004,2.254623e-07


In [7]:
# TOP 5000
top_5000_items_train = (
    data_train
    .groupby('item_id')
    .agg({'quantity': 'sum'})  # не уверен, что стоит считать записи, где quantity=0
    .query('quantity > 0')
    .nlargest(5000, 'quantity')
)

top_5000_items_train.head(5).join(pd.read_csv('product.csv').set_index('PRODUCT_ID'))

,quantity,MANUFACTURER,DEPARTMENT,BRAND,COMMODITY_DESC,SUB_COMMODITY_DESC,CURR_SIZE_OF_PRODUCT
item_id,,,,,,,
6534178,190227964,69,KIOSK-GAS,Private,COUPON/MISC ITEMS,GASOLINE-REG UNLEADED,
6533889,15978434,69,MISC SALES TRAN,Private,COUPON/MISC ITEMS,GASOLINE-REG UNLEADED,
6534166,12439291,69,MISC SALES TRAN,Private,COUPON/MISC ITEMS,GASOLINE-REG UNLEADED,
6544236,2501949,69,MISC SALES TRAN,Private,COUPON/MISC ITEMS,GASOLINE-REG UNLEADED,
1404121,1562004,69,KIOSK-GAS,Private,COUPON/MISC ITEMS,GASOLINE-REG UNLEADED,


In [8]:
# Генераторы случайных рекомендаций
random_gen = gen_inf_random_rec(
    item_weight_train.index,
    random_state=2021)

random_top5k_gen = gen_inf_random_rec(
    top_5000_items_train.index,
    random_state=2021)

weighted_random_gen = gen_inf_random_rec(
    item_weight_train.index,
    weights=item_weight_train['w_sum'],
    random_state=2021)

logweighted_random_gen = gen_inf_random_rec(
    item_weight_train.index,
    weights=item_weight_train['w_logsum'],
    random_state=2021)

In [9]:
# helper
def lslice(iterable, stop):
    """Get list of first n generator items"""
    return [itm for _, itm in zip(range(stop), iterable)]

In [10]:
%%time
result = (
    data_test
    .groupby('user_id')
    .agg(actual=('item_id', list))
    .assign(
        random_recommendation=lambda x: lslice(random_gen, x.shape[0]),
        random_top5k_recommendation=lambda x: lslice(random_top5k_gen, x.shape[0]),
        weighted_random_recommendation=lambda x: lslice(weighted_random_gen, x.shape[0]),
        logweighted_random_recommendation=lambda x: lslice(logweighted_random_gen, x.shape[0]),
    )
)
result.head(2)

Wall time: 5.38 s


,actual,random_recommendation,random_top5k_recommendation,weighted_random_recommendation,logweighted_random_recommendation
user_id,,,,,
1,"[821867, 834484, 856942, 865456, 889248, 90795...","[13910481, 9551712, 9552919, 7024844, 1370693]","[5567874, 908408, 965050, 869900, 888532]","[6534178, 12384737, 1098927, 967751, 1115387]","[9374013, 13672076, 2024965, 994928, 5576830]"
3,"[835476, 851057, 872021, 878302, 879948, 90963...","[965562, 314005, 6395990, 5125711, 978937]","[1077285, 9527290, 12262992, 915679, 962185]","[834117, 933835, 1046336, 958652, 6391045]","[819969, 951841, 1097350, 983584, 8091434]"


In [11]:
# Убедимся, что веса, реализованные как логарифм от суммы, практически не дают никакого преимущества популярным товарам
# В отличие от весов, пропорциональных непосредственно сумме
# Правда, неизвестно, что полезнее для бизнеса
n_top = 7
print(f"Top {n_top} повторяющихся случайных значений")
print(result['random_recommendation'].explode().value_counts().nlargest(n_top).to_dict())
print('*'*50)
print(f"Top {n_top} повторяющихся случайных значений, вес пропорционален log(sales_value + 1)")
print(result['logweighted_random_recommendation'].explode().value_counts().nlargest(n_top).to_dict())
print('*'*50)
print(f"Top {n_top} повторяющихся случайных значений, вес пропорционален sales_value")
print(result['weighted_random_recommendation'].explode().value_counts().nlargest(n_top).to_dict())

Top 7 повторяющихся случайных значений
{13877043: 4, 99340: 3, 13911498: 3, 926128: 3, 8068356: 3, 9655023: 3, 906602: 3}
**************************************************
Top 7 повторяющихся случайных значений, вес пропорционален log(sales_value + 1)
{1118028: 3, 913786: 3, 934399: 3, 927424: 3, 1119555: 3, 879689: 3, 1087581: 3}
**************************************************
Top 7 повторяющихся случайных значений, вес пропорционален sales_value
{6534178: 574, 6533889: 67, 1029743: 57, 1082185: 42, 916122: 39, 6533765: 39, 6534166: 36}


### Задание 2. Улучшение бейзлайнов и ItemItem

- Попробуйте улучшить бейзлайны, считая случаный на топ-5000 товаров (*это уже сделано в задании 1*)
- Попробуйте улучшить разные варианты ItemItemRecommender, выбирая число соседей $K$.

**Примечание** И здесь реализовал не так, как в методичке. Вместо pivot_table для создания матрицы user_id-item_id (заметно) быстрее использовать bsr_matrix. Далее поскольку процесс получения результатов для каждой модели одинаковый, можно реализовать цикл.

In [12]:
%%time
# Оставим только транзакции с товарами, входящими в Top-5000
user_item_df = (
    data_train[data_train['item_id'].isin(top_5000_items_train.index)]
    .filter(['user_id', 'item_id', 'quantity'])
)

# Мапинг user_id / item_id -> индекс строки / столбца матрицы и обратно
idx_to_userid = dict(enumerate(user_item_df['user_id'].unique()))
userid_to_idx = {item_id: idx for idx, item_id in idx_to_userid.items()}
idx_to_itemid = dict(enumerate(user_item_df['item_id'].unique()))
itemid_to_idx = {item_id: idx for idx, item_id in idx_to_itemid.items()}

user_item_matrix = bsr_matrix(
    (user_item_df['quantity'].astype(float),  # data
     (user_item_df['user_id'].map(userid_to_idx),  # row
      user_item_df['item_id'].map(itemid_to_idx))),  # col
    shape=(len(userid_to_idx), len(itemid_to_idx))).sign()  # оставляем 0 или 1

sparse_user_item = user_item_matrix.tocsr()
sparse_item_user = user_item_matrix.T.tocsr()

Wall time: 475 ms


In [13]:
# Имеем в виду, что есть пользователи, которые не приобретали товары из TOP-5000
# поэтому они не получают рекомендации item-item
print(np.setdiff1d(result.index, user_item_df['user_id']))

[ 650  729  954 1987 2364]


In [14]:
%%time
# Сравнение моделей
models = [
    ItemItemRecommender(K=3, num_threads=4),
    ItemItemRecommender(K=5, num_threads=4),
    ItemItemRecommender(K=7, num_threads=4),
    ItemItemRecommender(K=9, num_threads=4),
    CosineRecommender(K=5, num_threads=4),
    CosineRecommender(K=3, num_threads=4),
    TFIDFRecommender(K=3, num_threads=4),
    TFIDFRecommender(K=5, num_threads=4),
]

for model in models:
    model.fit(sparse_item_user, show_progress=False)
    rec_df = pd.DataFrame(
        ([[idx_to_itemid[rec] for rec, _ in
           model.recommend(userid=i,
                           user_items=sparse_user_item,   # на вход user-item matrix
                           N=5, # кол-во рекомендаций
                           filter_already_liked_items=False,
                           filter_items=None,
                           recalculate_user=True)]]
         for i in range(len(userid_to_idx))),
        index = userid_to_idx,
        columns=[f"{model.__class__.__name__}(K={model.K})"]
    )
    result = result.join(rec_df, how='inner')

Wall time: 8.77 s


### Результат сравнения моделей

In [15]:
# Precision@K
def precision_at_k(recommended_list, bought_list, k=5):
    _rec_list = recommended_list[:k]
    _b_and_r = np.intersect1d(bought_list, _rec_list)
    return _b_and_r.size / len(_rec_list)

def mean_precision_at_k(df, rec, bought, k=5):
    _result = df.apply(
        lambda row: precision_at_k(row[rec], row[bought], k),
        axis=1
    )
    return np.mean(_result)

In [16]:
precision_df = pd.DataFrame(
    ((baseline, mean_precision_at_k(result, baseline, 'actual'))
     for baseline in result.columns[1:]),
    columns=['baseline', 'precision@5']
)

precision_df.sort_values('precision@5', ascending=False)

,baseline,precision@5
4,ItemItemRecommender(K=3),0.170741
10,TFIDFRecommender(K=3),0.167861
7,ItemItemRecommender(K=9),0.166323
6,ItemItemRecommender(K=7),0.165538
9,CosineRecommender(K=3),0.161438
11,TFIDFRecommender(K=5),0.158370
5,ItemItemRecommender(K=5),0.156701
8,CosineRecommender(K=5),0.155916
2,weighted_random_recommendation,0.022189
1,random_top5k_recommendation,0.004418
